In [49]:
import bisect
from typing import List, Tuple

# open text file
def ReadFile(filename: str):
  with open(filename, 'r') as f:
    lines = f.readlines()
  return lines

class AlmanacMap(object):
  
  def __init__(self, sorted_from_values: list, map_from_value_to_to_value_and_count: dict):
    self._sorted_from_values = sorted_from_values
    self._map_from_value_to_to_value_and_count = map_from_value_to_to_value_and_count
    
  def Get(self, from_value: int):
    index = bisect.bisect(self._sorted_from_values, from_value)
    if index == 0:
      return from_value
    else:
      candidate_range_from_value = self._sorted_from_values[index - 1]
      candidate_range_end_value = candidate_range_from_value + self._map_from_value_to_to_value_and_count[candidate_range_from_value][1]
      if from_value >= candidate_range_from_value and from_value < candidate_range_end_value:
        return self._map_from_value_to_to_value_and_count[candidate_range_from_value][0] + (from_value - candidate_range_from_value)
      else:
        return from_value
      
  def _GetRange(self, from_value_range: Tuple) -> List[Tuple]:
    from_value_start, from_value_length = from_value_range
    from_value_end = from_value_start + from_value_length
    start_cursor = from_value_start
    from_ranges = []
    to_ranges = []
    while start_cursor < from_value_end:
      bisect_index = bisect.bisect(self._sorted_from_values, start_cursor)
      # Check if the start_cursor in the middle of a range.
      if bisect_index > 0:
        candidate_range_from_value = self._sorted_from_values[bisect_index - 1]
        candidate_range_end_value = candidate_range_from_value + self._map_from_value_to_to_value_and_count[candidate_range_from_value][1]
        if start_cursor >= candidate_range_from_value and start_cursor < candidate_range_end_value:
          from_ranges.append((start_cursor, candidate_range_end_value))
          to_ranges.append((self.Get(start_cursor), min(candidate_range_end_value, from_value_end) - start_cursor))
          if (self.Get(start_cursor) == 0):
            pass
          start_cursor = min(candidate_range_end_value, from_value_end)
          continue
      # Check if the start_cursor is in-between two ranges.
      if bisect_index < len(self._sorted_from_values):
        candidate_range_from_value = self._sorted_from_values[bisect_index]
        candidate_range_end_value = candidate_range_from_value + self._map_from_value_to_to_value_and_count[candidate_range_from_value][1]
        if start_cursor < candidate_range_from_value:
          from_ranges.append((start_cursor, candidate_range_from_value))
          to_ranges.append((self.Get(start_cursor), min(candidate_range_from_value, from_value_end) - start_cursor))
          if (self.Get(start_cursor) == 0):
            pass
          start_cursor = min(candidate_range_from_value, from_value_end)
          continue
      else:
        from_ranges.append((start_cursor, from_value_end))
        to_ranges.append((self.Get(start_cursor), from_value_end - start_cursor))
        if (self.Get(start_cursor) == 0):
          pass
        start_cursor = from_value_end
        continue
        
    return to_ranges
      
  def GetRange(self, from_value_ranges: List[Tuple]) -> List[Tuple]:
    result = []
    for from_value_range in from_value_ranges:
      result.extend(self._GetRange(from_value_range))
    return result
  
  def ParseInput(lines: list):
    from_values = []
    map_from_value_to_to_value_and_count = {}
    for line in lines:
      to_str, from_str, count_str = line.split(' ')
      # convert to int
      from_value = int(from_str)
      to_value = int(to_str)
      count_value = int(count_str)
      # populate from_values
      from_values.append(from_value)
      # populate map
      if from_value not in map_from_value_to_to_value_and_count:
        map_from_value_to_to_value_and_count[from_value] = (to_value, count_value)
    from_values.sort()
    return __class__(from_values, map_from_value_to_to_value_and_count)


class Almanac(object):
  
  MAP_NAMES = [
    'seed-to-soil', 
    'soil-to-fertilizer',
    'fertilizer-to-water',
    'water-to-light',
    'light-to-temperature',
    'temperature-to-humidity',
    'humidity-to-location'
  ]
  
  def __init__(self, seed_values: list, seed_ranges: list, map_of_maps: dict):
    self._seed_values = seed_values
    self._seed_ranges = seed_ranges
    self._map_of_maps = map_of_maps
    
  def _CalculateSeedLocation(self, seed_value: int):
    value = seed_value
    for map_name in __class__.MAP_NAMES:
      value = self._map_of_maps[map_name].Get(value)
    return value

  def _CalculateSeedRangeLocationRanges(self, seed_range: Tuple) -> List[Tuple]:
    value_ranges = [seed_range]
    for map_name in __class__.MAP_NAMES:
      value_ranges = self._map_of_maps[map_name].GetRange(value_ranges)

    return value_ranges
  
  def MinimumSeedRangeLocation(self):
    ranges = []
    for seed_range in self._seed_ranges:
      ranges.extend(self._CalculateSeedRangeLocationRanges(seed_range))
    # minimum first value
    return min([x[0] for x in ranges])
  
  def MinimumSeedLocation(self):
    return min([self._CalculateSeedLocation(seed_value) for seed_value in self._seed_values])

  def ParseInput(lines: list):
    index = 0
    
    # Parse the seeds.
    assert lines[index].startswith('seeds:')
    _, seeds = lines[index].split(':')
    seed_values = [int(x) for x in seeds.strip().split(' ')]
    # group seed values into 2 tuples
    seed_ranges = [(seed_values[i], seed_values[i + 1]) for i in range(0, len(seed_values), 2)]
    
    index += 1
    assert lines[index].strip() == ''
    index += 1
    
    # Parse the maps.
    map_of_maps = {}
    for map_name in __class__.MAP_NAMES:
      assert lines[index].startswith(map_name + ' map:')
      index += 1
      map_lines = []
      while index < len(lines) and lines[index].strip() != '':
        map_lines.append(lines[index])
        index += 1
      else:
        index += 1
      map_of_maps[map_name] = AlmanacMap.ParseInput(map_lines)
      
    return __class__(seed_values, seed_ranges, map_of_maps)

def SolvePartOne(input_file: str):
  lines = ReadFile(input_file)
  almanac = Almanac.ParseInput(lines)
  return almanac.MinimumSeedLocation()

def SolvePartTwo(input_file: str):
  lines = ReadFile(input_file)
  almanac = Almanac.ParseInput(lines)
  return almanac.MinimumSeedRangeLocation()

assert SolvePartOne('sample.txt') == 35
part_one_solution = SolvePartOne('input.txt')
print('Part One Solution:', part_one_solution)
assert part_one_solution == 510109797

assert SolvePartTwo('sample.txt') == 46
part_two_solution = SolvePartTwo('input.txt')
print('Part Two Solution:', part_two_solution)
assert part_two_solution == 9622622


Part One Solution: 510109797
Part Two Solution: 9622622
